In [ ]:
import tensorflow as tf
import h5py
import numpy as np
from sklearn.metrics import average_precision_score
import rna_model
import torch
import h5py
from torch.utils.data import DataLoader
from tqdm import tqdm
import glob


In [2]:
#pytorch load model
model_dir = glob.glob('/home/amber/multitask_RNA/wandb/run-*zai2tsbz/files/best_model.ckpt')[0]
model = rna_model.Splice_AI_2K((4,3000),(3,2000),32,0.001)
model.load_state_dict(torch.load(model_dir)['state_dict'])

<All keys matched successfully>

In [3]:
model.eval()

Splice_AI_2K(
  (conv1): Conv1d(4, 32, kernel_size=(1,), stride=(1,), padding=valid)
  (conv2): Conv1d(32, 32, kernel_size=(1,), stride=(1,), padding=valid)
  (block1): ResidualBlock(
    (block): Sequential(
      (0): ResidualUnit(
        (block): Sequential(
          (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Conv1d(32, 32, kernel_size=(11,), stride=(1,), padding=same)
          (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (4): ReLU()
          (5): Conv1d(32, 32, kernel_size=(11,), stride=(1,), padding=same)
        )
      )
      (1): ResidualUnit(
        (block): Sequential(
          (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): ReLU()
          (2): Conv1d(32, 32, kernel_size=(11,), stride=(1,), padding=same)
          (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_sta

In [16]:
model = torch.nn.Sequential(*list(model.children())[:-3])

In [14]:
%load_ext autoreload
%autoreload 2

import rna_model
from torchinfo import summary
import h5py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
model = rna_model.rbp_cnn(120,0.001)
summary(model,input_size=(32,4,2000))

Layer (type:depth-idx)                   Output Shape              Param #
rbp_cnn                                  [32, 120]                 --
├─Accuracy: 1-1                          --                        --
├─Accuracy: 1-2                          --                        --
├─Accuracy: 1-3                          --                        --
├─AveragePrecision: 1-4                  --                        --
├─AUROC: 1-5                             --                        --
├─Sequential: 1-6                        [32, 256, 2000]           --
│    └─Conv1d: 2-1                       [32, 256, 2000]           19,712
│    └─BatchNorm1d: 2-2                  [32, 256, 2000]           512
├─Sequential: 1-7                        [32, 256]                 --
│    └─MaxPool1d: 2-3                    [32, 256, 250]            --
│    └─Dropout: 2-4                      [32, 256, 250]            --
│    └─Conv1d: 2-5                       [32, 256, 250]            459,008
│    

In [23]:
model = rna_model.rbp_residual(120,0.001)
summary(model,input_size=(32,4,2000))

Layer (type:depth-idx)                   Output Shape              Param #
rbp_residual                             [32, 120]                 --
├─Accuracy: 1-1                          --                        --
├─Accuracy: 1-2                          --                        --
├─Accuracy: 1-3                          --                        --
├─AveragePrecision: 1-4                  --                        --
├─AUROC: 1-5                             --                        --
├─Sequential: 1-6                        [32, 256, 2000]           --
│    └─Conv1d: 2-1                       [32, 256, 2000]           19,712
│    └─BatchNorm1d: 2-2                  [32, 256, 2000]           512
├─Sequential: 1-7                        [32, 256]                 --
│    └─MaxPool1d: 2-3                    [32, 256, 250]            --
│    └─dilated_residual: 2-4             [32, 256, 250]            --
│    │    └─Sequential: 3-1              [32, 256, 250]            919,040
│    

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


model = MyModel()
model.fc2.register_forward_hook(get_activation('fc2'))
x = torch.randn(1, 25)
output = model(x)
print(activation['fc2'])